In [38]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import langchain
import torchvision
import torch.nn.functional as F
import kagglehub

In [2]:
torch.cuda.is_available()

True

In [6]:
# Download latest version
path = kagglehub.dataset_download("solarmainframe/ids-intrusion-csv")

Resuming download from 125829120 bytes (1591072049 bytes left)...
Resuming download to C:\Users\mjaye\.cache\kagglehub\datasets\solarmainframe\ids-intrusion-csv\1.archive (125829120/1716901169) bytes left.


100%|██████████| 1.60G/1.60G [02:18<00:00, 11.5MB/s]

Extracting files...


Path to dataset files: C:\Users\mjaye\.cache\kagglehub\datasets\solarmainframe\ids-intrusion-csv\versions\1


C:\Users\mjaye\PycharmProjects\TRACE
exists: False
isfile: False
readable: False


<bound method DataFrame.info of          Dst Port  Protocol            Timestamp  Flow Duration  Tot Fwd Pkts  \
0             443         6  02/03/2018 08:47:38         141385             9   
1           49684         6  02/03/2018 08:47:38            281             2   
2             443         6  02/03/2018 08:47:40         279824            11   
3             443         6  02/03/2018 08:47:40            132             2   
4             443         6  02/03/2018 08:47:41         274016             9   
...           ...       ...                  ...            ...           ...   
1048570      3389         6  02/03/2018 02:08:18        3982183            14   
1048571      3389         6  02/03/2018 02:08:22        3802316            14   
1048572      3389         6  02/03/2018 02:08:25        4004239            14   
1048573      3389         6  02/03/2018 02:08:29        3998435            14   
1048574      3389         6  02/03/2018 02:08:33        3972651            14

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import (classification_report, confusion_matrix, accuracy_score,
                            precision_recall_fscore_support)
from sklearn.neighbors import NearestNeighbors
from sklearn.utils.class_weight import compute_class_weight
warnings.filterwarnings('ignore')

import time
import gc
import os
from collections import Counter
import pickle

In [3]:
path = "C:\\Users\\mjaye\\PycharmProjects\\TRACE\\solarmainframe\\ids-intrusion-csv\\versions\\1"

In [4]:
def _create_memmap_windows_safe(filename, shape, dtype):
        if os.path.exists(filename):
            os.remove(filename)

        dtype = np.dtype(dtype)
        bytes_needed = int(np.prod(shape)) * dtype.itemsize

        # Pre-allocate file size
        with open(filename, "wb") as f:
            f.seek(bytes_needed - 1)
            f.write(b"\0")

        # Map existing file
        return np.memmap(filename, dtype=dtype, mode="r+", shape=shape)

In [5]:
SEED = 42
np.random.seed(SEED)

class FullDatasetLoader:
    """Loads ALL 10 CSV files with ALL features using memory mapping"""
    
    def __init__(self, data_path, chunk_size=100000):
        self.data_path = data_path
        self.chunk_size = chunk_size
        self.scaler = None
        self.label_encoder = None
        self.feature_names = None
        self.num_features = None
        
    def load_full_dataset(self):
        """Load ALL 10 CSV files with ALL features"""
        
        print("\n" + "="*80)
        print("LOADING FULL DATASET (ALL 10 CSV FILES)")
        print("="*80)
        
        csv_files = sorted([f for f in os.listdir(self.data_path) if f.endswith('.csv')])
        print(f"Found {len(csv_files)} CSV files")
        print(f"Files: {csv_files}")
        
        # First pass: collect metadata and determine feature names
        print("\n Pass 1: Collecting metadata from all files...")
        all_columns = None
        total_rows = 0
        
        for i, file in enumerate(csv_files, 1):
            filepath = os.path.join(self.data_path, file)
            print(f"Scanning {i}/{len(csv_files)}: {file}")
            
            # Read just first chunk to get columns
            df_sample = pd.read_csv(filepath, nrows=1000)
            df_sample = df_sample[df_sample['Label'] != 'Label']
            
            if all_columns is None:
                all_columns = df_sample.columns.tolist()
            
            # Count total rows
            row_count = sum(1 for _ in open(filepath)) - 1  # -1 for header
            total_rows += row_count
            print(f"  Rows: {row_count:,}")
        
        print(f"\n Total rows across all files: {total_rows:,}")
        print(f" Total columns: {len(all_columns)}")
        
        # Remove non-feature columns
        cols_to_remove = ['Timestamp', 'Flow ID', 'Source IP', 'Destination IP', 
                         'Source Port', 'Destination Port', 'Protocol']
        feature_cols = [c for c in all_columns if c not in cols_to_remove and c != 'Label']
        
        self.feature_names = feature_cols
        self.num_features = len(feature_cols)
        
        print(f" Feature columns (ALL): {self.num_features}")
        print(f" Using ALL features - NO feature selection")
        
        # Second pass: Load all data in chunks and save to memory-mapped file
        print("\n Pass 2: Loading all data and creating memory-mapped arrays...")
        
        # Create temporary memory-mapped files
        X_memmap = _create_memmap_windows_safe("X_temp.dat", (total_rows, self.num_features), np.float32)
        y_memmap = _create_memmap_windows_safe("y_temp.dat", (total_rows,), np.uint8)
        current_idx = 0
        
        for i, file in enumerate(csv_files, 1):
            filepath = os.path.join(self.data_path, file)
            print(f"\n{'='*60}")
            print(f"Processing {i}/{len(csv_files)}: {file}")
            print(f"{'='*60}")
            
            chunk_iter = pd.read_csv(filepath, chunksize=self.chunk_size)
            file_rows = 0
            
            for chunk_num, chunk in enumerate(chunk_iter):
                # Clean chunk
                chunk = chunk[chunk['Label'] != 'Label']
                
                if len(chunk) == 0:
                    continue
                
                # Drop non-feature columns
                chunk = chunk.drop(columns=[c for c in cols_to_remove if c in chunk.columns], 
                                  errors='ignore')
                
                # Separate features and labels
                labels = chunk['Label'].astype(str).str.strip()
                y_chunk = (labels != 'BENIGN').astype(np.uint8)  # 0 benign, 1 attack
                X_chunk = chunk[feature_cols]
                
                # Convert to numeric
                X_chunk = X_chunk.apply(pd.to_numeric, errors='coerce')
                X_chunk = X_chunk.replace([np.inf, -np.inf], np.nan)
                X_chunk = X_chunk.fillna(0)
                
                # Store in memmap
                n_samples = len(X_chunk)
                X_memmap[current_idx:current_idx + n_samples] = X_chunk.values.astype(np.float32)
                y_memmap[current_idx:current_idx + n_samples] = y_chunk
                
                current_idx += n_samples
                file_rows += n_samples
                
                if chunk_num % 10 == 0:
                    print(f"  Chunk {chunk_num}: Processed {current_idx:,}/{total_rows:,} rows")
                
                # Clean up
                del chunk, X_chunk, y_chunk
                gc.collect()
            
            print(f" File {i} complete: {file_rows:,} rows processed")
        
        print(f"\n Total rows loaded: {current_idx:,}")
        
        # Trim to actual size
        X_final = X_memmap[:current_idx]
        y_final = y_memmap[:current_idx]
        
        # Flush to disk
        X_final.flush()
        y_final.flush()
        
        return X_final, y_final, current_idx
    
    def fit_scaler_on_memmap(self, X_memmap, y_memmap, total_samples):
        """Fit scaler incrementally on memory-mapped data"""
        self.scaler = StandardScaler()
        chunk_size = 100000
        for i in range(0, total_samples, chunk_size):
            end = min(i + chunk_size, total_samples)
            Xc = X_memmap[i:end]
            yc = y_memmap[i:end]
            benign_mask = (yc == 0)
            if benign_mask.any():
                self.scaler.partial_fit(Xc[benign_mask])
        return self.scaler
    
    def transform_and_save_windows_safe(self, X_memmap, total_samples, out_file="X_scaled.dat"):
        print("\n Scaling ALL data...")

        # Remove old file if it exists (avoids “invalid argument” / locking issues)
        if os.path.exists(out_file):
            os.remove(out_file)

        total_samples = int(total_samples)
        num_features = int(self.num_features)
        dtype = np.float32
        bytes_needed = total_samples * num_features * np.dtype(dtype).itemsize

        # Pre-allocate file to exact size
        with open(out_file, "wb") as f:
            f.seek(bytes_needed - 1)
            f.write(b"\0")

        # Now memmap with r+ (read/write existing)
        X_scaled = np.memmap(out_file, dtype=dtype, mode="r+", shape=(total_samples, num_features))

        chunk_size = 100000
        for i in range(0, total_samples, chunk_size):
            end_idx = min(i + chunk_size, total_samples)
            chunk = X_memmap[i:end_idx]
            X_scaled[i:end_idx] = self.scaler.transform(chunk).astype(dtype)

            if i % 500000 == 0:
                print(f"  Scaled {i:,}/{total_samples:,} samples...")

        X_scaled.flush()
        print(" All data scaled and saved to", out_file)
        return X_scaled

In [6]:
dataset_loader = FullDatasetLoader(data_path=path, chunk_size=100000)
X_raw, y, n = dataset_loader.load_full_dataset()
print(X_raw.shape, y.shape, n)


LOADING FULL DATASET (ALL 10 CSV FILES)
Found 10 CSV files
Files: ['02-14-2018.csv', '02-15-2018.csv', '02-16-2018.csv', '02-20-2018.csv', '02-21-2018.csv', '02-22-2018.csv', '02-23-2018.csv', '02-28-2018.csv', '03-01-2018.csv', '03-02-2018.csv']

 Pass 1: Collecting metadata from all files...
Scanning 1/10: 02-14-2018.csv
  Rows: 1,048,575
Scanning 2/10: 02-15-2018.csv
  Rows: 1,048,575
Scanning 3/10: 02-16-2018.csv
  Rows: 1,048,575
Scanning 4/10: 02-20-2018.csv
  Rows: 7,948,748
Scanning 5/10: 02-21-2018.csv
  Rows: 1,048,575
Scanning 6/10: 02-22-2018.csv
  Rows: 1,048,575
Scanning 7/10: 02-23-2018.csv
  Rows: 1,048,575
Scanning 8/10: 02-28-2018.csv
  Rows: 613,104
Scanning 9/10: 03-01-2018.csv
  Rows: 331,125
Scanning 10/10: 03-02-2018.csv
  Rows: 1,048,575

 Total rows across all files: 16,233,002
 Total columns: 80
 Feature columns (ALL): 77
 Using ALL features - NO feature selection

 Pass 2: Loading all data and creating memory-mapped arrays...

Processing 1/10: 02-14-2018.csv

In [7]:
X_raw.flush()
y.flush()
print("Saved/Flushed:", X_raw.filename, y.filename)

Saved/Flushed: C:\Users\mjaye\PycharmProjects\TRACE\X_temp.dat C:\Users\mjaye\PycharmProjects\TRACE\y_temp.dat


In [8]:
import os, gc

x_path = X_raw.filename
y_path = y.filename

# close memmaps cleanly
del X_raw
del y
gc.collect()

# rename (move) to stable names
os.rename(x_path, "X_raw_final.dat")
os.rename(y_path, "y_final.dat")

print("Renamed to X_raw_final.dat and y_final.dat")

Renamed to X_raw_final.dat and y_final.dat


In [11]:
import numpy as np

n = 16233002          # your saved n
num_features = 77     # your saved feature count

X_raw = np.memmap("X_raw_final.dat", dtype="float32", mode="r", shape=(n, num_features))
y     = np.memmap("y_final.dat",     dtype="uint8",   mode="r", shape=(n,))
print(X_raw.shape, y.shape)

(16233002, 77) (16233002,)


In [12]:
print("X:", X_raw.shape, X_raw.dtype)
print("y:", y.shape, y.dtype)
print("benign:", int((y==0).sum()), "attack:", int((y==1).sum()))
print("X min/max (sample):", float(X_raw[:10000].min()), float(X_raw[:10000].max()))

X: (16233002, 77) float32
y: (16233002,) uint8
benign: 59 attack: 16232943
X min/max (sample): -1.0 112642176.0


In [13]:
import pandas as pd, os

one_file = os.path.join(path, "02-14-2018.csv")
s = pd.read_csv(one_file, usecols=["Label"]).iloc[:200000, 0].astype(str)

print("raw uniques (top):")
print(s.value_counts().head(20))

print("\nnormalized uniques (top):")
sn = s.str.strip().str.upper()
print(sn.value_counts().head(20))

raw uniques (top):
Label
FTP-BruteForce    179244
SSH-Bruteforce     20490
Benign               266
Name: count, dtype: int64

normalized uniques (top):
Label
FTP-BRUTEFORCE    179244
SSH-BRUTEFORCE     20490
BENIGN               266
Name: count, dtype: int64


In [14]:
import numpy as np, os, gc, pandas as pd

csv_files = sorted([f for f in os.listdir(path) if f.endswith(".csv")])

# create fresh y memmap
y_correct = np.memmap("y_correct.dat", dtype="uint8", mode="w+", shape=(n,))

current_idx = 0
chunk_size = 100_000

for i, file in enumerate(csv_files, 1):
    fp = os.path.join(path, file)
    print(f"Rebuilding y {i}/{len(csv_files)}: {file}")

    for chunk in pd.read_csv(fp, usecols=["Label"], chunksize=chunk_size):
        # remove repeated header rows if any
        lab = chunk["Label"].astype(str)
        lab = lab[lab != "Label"]

        if len(lab) == 0:
            continue

        lab_norm = lab.str.strip().str.upper()

        # benign = exactly BENIGN after normalization
        y_chunk = (lab_norm != "BENIGN").astype(np.uint8).values

        end = current_idx + len(y_chunk)
        y_correct[current_idx:end] = y_chunk
        current_idx = end

    print("  current_idx:", f"{current_idx:,}")

y_correct.flush()
print("DONE. wrote:", current_idx, "expected:", n)


Rebuilding y 1/10: 02-14-2018.csv
  current_idx: 1,048,575
Rebuilding y 2/10: 02-15-2018.csv
  current_idx: 2,097,150
Rebuilding y 3/10: 02-16-2018.csv
  current_idx: 3,145,724
Rebuilding y 4/10: 02-20-2018.csv
  current_idx: 11,094,472
Rebuilding y 5/10: 02-21-2018.csv
  current_idx: 12,143,047
Rebuilding y 6/10: 02-22-2018.csv
  current_idx: 13,191,622
Rebuilding y 7/10: 02-23-2018.csv
  current_idx: 14,240,197
Rebuilding y 8/10: 02-28-2018.csv
  current_idx: 14,853,268
Rebuilding y 9/10: 03-01-2018.csv
  current_idx: 15,184,368
Rebuilding y 10/10: 03-02-2018.csv
  current_idx: 16,232,943
DONE. wrote: 16232943 expected: 16233002


In [15]:
y = np.memmap("y_correct.dat", dtype="uint8", mode="r", shape=(n,))
print("benign:", int((y==0).sum()), "attack:", int((y==1).sum()))

benign: 13484767 attack: 2748235


In [16]:
idx = np.arange(n)
train_benign_idx = idx[y == 0] 

In [17]:
from sklearn.model_selection import train_test_split
import numpy as np

idx = np.arange(n)
benign_idx = idx[y == 0]
attack_idx = idx[y == 1]

# Split benign into train + benign_holdout
train_benign_idx, benign_holdout_idx = train_test_split(
    benign_idx, test_size=0.30, random_state=42, shuffle=True
)

# Val/Test = benign_holdout + all attacks (then split)
val_idx, test_idx = train_test_split(
    np.concatenate([benign_holdout_idx, attack_idx]),
    test_size=0.50, random_state=42, shuffle=True
)

print("train benign:", len(train_benign_idx))
print("val:", len(val_idx), "attack%:", float((y[val_idx]==1).mean()))
print("test:", len(test_idx), "attack%:", float((y[test_idx]==1).mean()))

np.save("train_benign_idx.npy", train_benign_idx)
np.save("val_idx.npy", val_idx)
np.save("test_idx.npy", test_idx)


train benign: 9439336
val: 3396833 attack%: 0.4045559496154212
test: 3396833 attack%: 0.404502075904232


In [18]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
chunk = 100_000

train_sorted = np.sort(train_benign_idx)  # makes sequential reads faster

for start in range(0, len(train_sorted), chunk):
    sel = train_sorted[start:start+chunk]
    scaler.partial_fit(X_raw[sel])

import joblib
joblib.dump(scaler, "scaler_train_benign.pkl")
print("scaler fitted")

scaler fitted


In [19]:
import os, numpy as np

out_file = "X_scaled.dat"
if os.path.exists(out_file):
    os.remove(out_file)

bytes_needed = int(n) * int(num_features) * np.dtype(np.float32).itemsize
with open(out_file, "wb") as f:
    f.seek(bytes_needed - 1)
    f.write(b"\0")

X_scaled = np.memmap(out_file, dtype="float32", mode="r+", shape=(n, num_features))

chunk_size = 100_000
for i in range(0, n, chunk_size):
    end = min(i + chunk_size, n)
    X_scaled[i:end] = scaler.transform(X_raw[i:end]).astype(np.float32)
    if i % 500_000 == 0:
        print(f"scaled {i:,}/{n:,}")

X_scaled.flush()
print("X_scaled:", X_scaled.shape)

scaled 0/16,233,002
scaled 500,000/16,233,002
scaled 1,000,000/16,233,002
scaled 1,500,000/16,233,002
scaled 2,000,000/16,233,002
scaled 2,500,000/16,233,002
scaled 3,000,000/16,233,002
scaled 3,500,000/16,233,002
scaled 4,000,000/16,233,002
scaled 4,500,000/16,233,002
scaled 5,000,000/16,233,002
scaled 5,500,000/16,233,002
scaled 6,000,000/16,233,002
scaled 6,500,000/16,233,002
scaled 7,000,000/16,233,002
scaled 7,500,000/16,233,002
scaled 8,000,000/16,233,002
scaled 8,500,000/16,233,002
scaled 9,000,000/16,233,002
scaled 9,500,000/16,233,002
scaled 10,000,000/16,233,002
scaled 10,500,000/16,233,002
scaled 11,000,000/16,233,002
scaled 11,500,000/16,233,002
scaled 12,000,000/16,233,002
scaled 12,500,000/16,233,002
scaled 13,000,000/16,233,002
scaled 13,500,000/16,233,002
scaled 14,000,000/16,233,002
scaled 14,500,000/16,233,002
scaled 15,000,000/16,233,002
scaled 15,500,000/16,233,002
scaled 16,000,000/16,233,002
X_scaled: (16233002, 77)


In [20]:
import torch
from torch.utils.data import Dataset, DataLoader

class MemmapDataset(Dataset):
    def __init__(self, X, y, indices):
        self.X = X
        self.y = y
        self.indices = np.asarray(indices)

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, i):
        idx = self.indices[i]
        x = torch.from_numpy(self.X[idx].copy())   # copy avoids memmap view quirks
        y = torch.tensor(int(self.y[idx]), dtype=torch.long)
        return x, y

train_ds = MemmapDataset(X_scaled, y, train_benign_idx)
val_ds   = MemmapDataset(X_scaled, y, val_idx)
test_ds  = MemmapDataset(X_scaled, y, test_idx)

train_loader = DataLoader(train_ds, batch_size=512, shuffle=True, num_workers=0, pin_memory=True)
val_loader   = DataLoader(val_ds, batch_size=512, shuffle=False, num_workers=0, pin_memory=True)
test_loader  = DataLoader(test_ds, batch_size=512, shuffle=False, num_workers=0, pin_memory=True)

In [77]:
input_dim = X_scaled.shape[1]
hidden_dim = min(512, max(128, 2 * input_dim))
latent_dim = max(8, min(32, input_dim // 4))

In [78]:
@torch.no_grad()
def evaluate(model, loader, loss_fn, device):
    model.eval()
    total, n = 0.0, 0

    for xb, _ in loader:
        xb = xb.to(device, non_blocking=True)
        recon, mu, logvar = model(xb)

        loss_val, _, _ = loss_fn(xb, recon, mu, logvar)

        bs = xb.size(0)
        total += loss_val.item() * bs
        n += bs

    return total / n


def fit(model, train_loader, val_loader, loss_fn, optimizer, device, epochs=100, patience=5):
    best_val_loss = float("inf")
    bad = 0

    for epoch in range(epochs):
        model.train()
        total, n = 0.0, 0

        for xb, _ in train_loader:
            xb = xb.to(device, non_blocking=True)
            recon, mu, logvar = model(xb)

            loss_val, recon_l, kld = loss_fn(xb, recon, mu, logvar)

            optimizer.zero_grad(set_to_none=True)
            loss_val.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)
            optimizer.step()

            bs = xb.size(0)
            total += loss_val.item() * bs
            n += bs

        train_loss = total / n
        val_loss = evaluate(model, val_loader, loss_fn, device)

        print(f"Epoch {epoch+1}/{epochs} | train_loss={train_loss:.4f} | val_loss={val_loss:.4f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            bad = 0
            torch.save(model.state_dict(), "best_vae.pth")
        else:
            bad += 1
            if bad >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                model.load_state_dict(torch.load("best_vae.pth", map_location=device))
                break

In [79]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [80]:
device

device(type='cuda')

xb finite: True
recon finite: False
mu finite: False
logvar finite: False
exp(logvar) finite: False min/max: nan nan
t finite: False t min/max: nan nan


Epoch 1/50 | train_loss=nan | val_loss=nan


KeyboardInterrupt: 

In [91]:
import importlib, inspect, VAE_Model
importlib.reload(VAE_Model)

print("IMPORTING:", VAE_Model.__file__)
print(inspect.getsource(VAE_Model.Model.encode))
print(inspect.getsource(VAE_Model.Model.reparameterize))

IMPORTING: C:\Users\mjaye\PycharmProjects\TRACE\VAE_Model.py
    def encode(self, x):
        h = self.encoder(x)
        mu = self.mean_layer(h)
        logvar = self.logvar_layer(h)
        logvar = torch.clamp(logvar, -10.0, 10.0)  # stabilizer
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + std * eps



In [92]:
import torch

model = VAE_Model.Model(input_dim=77, hidden_dim=256, latent_dim=16).to(device)

# Check model params
bad = []
for name, p in model.named_parameters():
    if not torch.isfinite(p).all():
        bad.append(name)

print("Non-finite params:", bad[:20], "count=", len(bad))

Non-finite params: [] count= 0


In [93]:
xb, _ = next(iter(train_loader))
xb = xb.to(device)

x = xb
for i, layer in enumerate(model.encoder):
    x = layer(x)
    if not torch.isfinite(x).all():
        print("Non-finite after encoder layer", i, layer)
        print("min/max:", x.min().item(), x.max().item())
        break

mu = model.mean_layer(x)
logvar = model.logvar_layer(x)
print("mu finite:", torch.isfinite(mu).all().item())
print("logvar finite:", torch.isfinite(logvar).all().item())
print("logvar min/max:", logvar.min().item(), logvar.max().item())

mu finite: True
logvar finite: True
logvar min/max: -0.8977354168891907 0.5919608473777771


In [94]:
xb, _ = next(iter(train_loader))
xb = xb.to(device)

recon, mu, logvar = model(xb)

print("xb finite:", torch.isfinite(xb).all().item())
print("recon finite:", torch.isfinite(recon).all().item())
print("mu finite:", torch.isfinite(mu).all().item())
print("logvar finite:", torch.isfinite(logvar).all().item())

exp_logvar = torch.exp(logvar)
t = 1 + logvar - mu.pow(2) - exp_logvar   # the inside of KL

print("exp(logvar) finite:", torch.isfinite(exp_logvar).all().item(),
      "min/max:", exp_logvar.min().item(), exp_logvar.max().item())
print("t finite:", torch.isfinite(t).all().item(),
      "t min/max:", t.min().item(), t.max().item())

xb finite: True
recon finite: True
mu finite: True
logvar finite: True
exp(logvar) finite: True min/max: 0.20704416930675507 4.005405902862549
t finite: True t min/max: -2.0831148624420166 -1.7881393432617188e-07


In [101]:
N_TRAIN = 500_000   # try 200k / 500k / 1M / 2M

train_benign_idx_small = np.random.choice(train_benign_idx, size=N_TRAIN, replace=False)

train_ds = MemmapDataset(X_scaled, y, train_benign_idx_small)
train_loader = DataLoader(train_ds, batch_size=1024, shuffle=True, pin_memory=True, num_workers=0)

print("steps/epoch:", len(train_loader))

steps/epoch: 489


In [102]:
import VAE_Model

model = VAE_Model.Model(input_dim=77, hidden_dim=256, latent_dim=16).to(device)
loss_fn = VAE_Model.TotalLoss()
opt = torch.optim.Adam(model.parameters(), lr=3e-4)

In [105]:
# split val indices
val_benign_idx = val_idx[y[val_idx] == 0]
val_attack_idx = val_idx[y[val_idx] == 1]

val_benign_loader = DataLoader(MemmapDataset(X_scaled, y, val_benign_idx),
                               batch_size=2048, shuffle=False, pin_memory=True, num_workers=0)

val_attack_loader = DataLoader(MemmapDataset(X_scaled, y, val_attack_idx),
                               batch_size=2048, shuffle=False, pin_memory=True, num_workers=0)

print("val benign loss:", evaluate(model, val_benign_loader, loss_fn, device))
print("val attack loss:", evaluate(model, val_attack_loader, loss_fn, device))

val benign loss: 12.347293849225787
val attack loss: 17806.91465356652


In [107]:
fit(model, train_loader, val_benign_loader, loss_fn, opt, device=device, epochs=50, patience=20)

Epoch 1/50 | train_loss=48.5335 | val_loss=11.9703
Epoch 2/50 | train_loss=48.4268 | val_loss=13.0517
Epoch 3/50 | train_loss=47.8888 | val_loss=11.8225
Epoch 4/50 | train_loss=47.9732 | val_loss=12.0997
Epoch 5/50 | train_loss=48.0023 | val_loss=12.1724
Epoch 6/50 | train_loss=47.6889 | val_loss=11.7586
Epoch 7/50 | train_loss=47.7345 | val_loss=11.6343
Epoch 8/50 | train_loss=50.7595 | val_loss=12.1623
Epoch 9/50 | train_loss=48.1266 | val_loss=12.6220
Epoch 10/50 | train_loss=48.0012 | val_loss=11.7990
Epoch 11/50 | train_loss=47.7002 | val_loss=12.9412
Epoch 12/50 | train_loss=47.4624 | val_loss=12.1714
Epoch 13/50 | train_loss=47.4898 | val_loss=13.6290
Epoch 14/50 | train_loss=48.2797 | val_loss=12.6038
Epoch 15/50 | train_loss=47.4027 | val_loss=11.5423
Epoch 16/50 | train_loss=47.4362 | val_loss=12.4905
Epoch 17/50 | train_loss=47.4856 | val_loss=11.7703
Epoch 18/50 | train_loss=47.5653 | val_loss=11.8547
Epoch 19/50 | train_loss=47.3064 | val_loss=12.2019
Epoch 20/50 | train_l